In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
582,This movie is up there with the all-time class...,positive
409,I really really love this show! I have always ...,positive
353,I thought this had the right blend of characte...,positive
27,I can't get over how lazy the director is with...,negative
751,This is one of the best and moodiest Vampire T...,positive


In [10]:
# data preprocessing
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vinayak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vinayak/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
df = normalize_text(df)
df.head()

,review,sentiment
582,movie time classic music camera shot acting ex...,positive
409,really really love show always liked show spac...,positive
353,thought right blend character plot futuristic ...,positive
27,get lazy director movie instead setting scene ...,negative
751,one best moodiest vampire tale ever love movie...,positive


In [11]:
df['sentiment'].value_counts()

sentiment
negative    260
positive    240
Name: count, dtype: int64

In [12]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [13]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
582,movie time classic music camera shot acting ex...,1
409,really really love show always liked show spac...,1
353,thought right blend character plot futuristic ...,1
27,get lazy director movie instead setting scene ...,0
751,one best moodiest vampire tale ever love movie...,1


In [14]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [15]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [22]:
import dagshub

mlflow.set_tracking_uri("https://dagshub.com/Vinayakmane47/imdb_mlops.mlflow/")
dagshub.init(repo_owner='vinayak', repo_name='IMBD_MLOPS', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Repository IMBD_MLOPS doesn't exist, creating it under current user.

Initialized MLflow to track repo "vinayak/IMBD_MLOPS"

Repository vinayak/IMBD_MLOPS initialized!

2025/11/11 01:36:42 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/a880e22a6b2f43ba8f1f625771a8c627', creation_time=1762785402809, experiment_id='0', last_update_time=1762785402809, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [23]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-11-11 01:37:07,830 - INFO - Starting MLflow run...
2025-11-11 01:37:08,436 - INFO - Logging preprocessing parameters...
2025-11-11 01:37:09,423 - INFO - Initializing Logistic Regression model...
2025-11-11 01:37:09,424 - INFO - Fitting the model...
2025-11-11 01:37:09,526 - INFO - Model training complete.
2025-11-11 01:37:09,526 - INFO - Logging model parameters...
2025-11-11 01:37:09,800 - INFO - Making predictions...
2025-11-11 01:37:09,801 - INFO - Calculating evaluation metrics...
2025-11-11 01:37:09,810 - INFO - Logging evaluation metrics...
2025-11-11 01:37:11,064 - INFO - Saving and logging the model...
2025/11/11 01:37:19 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-11-11 01:37:19,406 - INFO - Model training and logging completed in 10.97 seconds.
2025-11-11 01:37:19,407 - INFO - Accuracy: 0.616
2025-11-11 01:37:19,408 - INFO - Precision: 0.5522388059701493
2025-11